# 미해결 버그

### Q. backend 토큰 무효화

In [ ]:
간단히 말해 실패했다.

### Q. CSRF 토큰

In [ ]:
무시하는 방향으로 해결했다.

# 해결된 버그

## Nginx

### Q. Nginx에서 javascript나 이미지파일, css가 안불러와진다

A. /etc/nginx/sites-available/default의 sever 내부에 location ~*.확장자로 경로를 지정한다

In [ ]:
server_name 43.202.61.123;

location / {
    root /home/chicken/ChatGPT-DRF-FE/HTML;
    index main.html;
}

location ~* \.(js|css|gif|png|jpg|jpeg|webp|svg|ico)$ {
    root /home/chicken/ChatGPT-DRF-FE;
}

### Q. 주소 매핑하는법

A. location에 주소매핑을 추가하고 root대신 alias로 폴더명을 지정한다.

In [ ]:
location /accounts/login {
    alias /home/chicken/ChatGPT-DRF-FE/HTML;
    index login.html;
}

location /accounts/register {
    alias /home/chicken/ChatGPT-DRF-FE/HTML;
    index register.html;
}

location /accounts/logout {
    alias /home/chicken/ChatGPT-DRF-FE/HTML;
    index logout.html;
}

location /quiz {
    alias /home/chicken/ChatGPT-DRF-FE/HTML;
    index quiz.html;
}

### Q. 403 Forbidden

A. 권한 문제이다. 이번에는 가장 위의 사용자폴더의 권한을 755로 변경해주어서 해결했다.

## JavaScript

### Q. Javascript에서 html 임포트하는법

A. Javascript를 이용해 임포트했다.

In [ ]:
(function () {
    function includeHtml() {
        const includeTarget = document.querySelectorAll(".includeJs");
        includeTarget.forEach(function (el, idx) {
            const targetFile = el.dataset.includeFile;
            if (targetFile) {
                let xhttp = new XMLHttpRequest();

                xhttp.onreadystatechange = function () {
                    if (this.readyState === XMLHttpRequest.DONE) {
                        this.status === 200 ? (el.innerHTML = this.responseText) : null;
                        this.status === 404 ? (el.innerHTML = "include not found.") : null;
                    }
                };
                xhttp.open("GET", targetFile, true);
                xhttp.send();
                return;
            }
        });
    }

    includeHtml();
})();

### Q. javascript 여러개를 묶는법

A. 이 역시 base javascript를 사용했다.

In [ ]:
const script1 = document.createElement('script');
script1.src = '/JavaScript/include.js';
document.head.appendChild(script1);

const script2 = document.createElement('script');
script2.src = '/JavaScript/header.js';
document.head.appendChild(script2);

const script3 = document.createElement('script');
script3.src = '/JavaScript/logout.js';
document.head.appendChild(script3);

const script4 = document.createElement('script');
script4.src = '/JavaScript/token.js';
document.head.appendChild(script4);


### Q. 한 자바스크립트에서 사용한 변수를 다른 자바스크립트에서 충돌하는경우

A. 함수로 감싸고 const변수로 지정한 다음에 함수를 실행하는것으로 변수를 휘발시켰다.

### Q. header를 import했더니 로드 당시에는 헤더가없어서 작동하지 않는 버그

A. 일정시간 이후 재싱행함으로서 리로드하는 방법으로 해결했다.

In [ ]:
맨 처음에는 DOMContentLoaded를 이용해서 하려고했으나, 정상적으로 작동하지않았다.
그리고 시간텀을 두지 않고 재시작하니 무한 재시작의 회로에 걸려서 로드하는데 엄청난 렉이 동반되었다.
또한 새로고침은 별로 도움이 되지 않았다. 이 역시 렉이걸리고 여러번 걸릴 가능성이 컸다.
결국 함수화한다음 재귀의 형태로 일정시간 이후 리로드하는것으로 변경하였다.

스크립트 간의 비동기통신 역시 생각해봤지만 한개 문서안에 다 있어야하기떄문에 나눠놓기 원하는 내 목적과는 달라서 반려돠었다.

## DJango

### Q. type object 'UserCreateAPIView' has no attribute 'get_extra_actions'

A. router = DefaultRouter() 대신 views.UserCreateAPIView.as_view()로 매핑한다

### Q. 비밀번호 해싱 오류

A. serializers에서 create함수를 만들어서 password를 바꾸면 된다.

In [ ]:
def create(self, validated_data):
    '''
    직렬화할떄 비밀번호를 해싱하는작업.
    '''
    user = super().create(validated_data)
    user.set_password(validated_data['password'])
    user.save()
    return user

### Q. User 커스텀 클래스에서 related_name를 요구한다

In [ ]:
ERRORS:
accounts.User.groups: (fields.E304) Reverse accessor 'Group.user_set' for 'accounts.User.groups' clashes with reverse accessor for 'auth.User.groups'.
        HINT: Add or change a related_name argument to the definition for 'accounts.User.groups' or 'auth.User.groups'.
accounts.User.user_permissions: (fields.E304) Reverse accessor 'Permission.user_set' for 'accounts.User.user_permissions' clashes with reverse accessor for 'auth.User.user_permissions'.
        HINT: Add or change a related_name argument to the definition for 'accounts.User.user_permissions' or 'auth.User.user_permissions'.
auth.User.groups: (fields.E304) Reverse accessor 'Group.user_set' for 'auth.User.groups' clashes with reverse accessor for 'accounts.User.groups'.
        HINT: Add or change a related_name argument to the definition for 'auth.User.groups' or 'accounts.User.groups'.
auth.User.user_permissions: (fields.E304) Reverse accessor 'Permission.user_set' for 'auth.User.user_permissions' clashes with reverse accessor for 'accounts.User.user_permissions'.
        HINT: Add or change a related_name argument to the definition for 'auth.User.user_permissions' or 'accounts.User.user_permissions'.

A. 새로 테이블을 만들어서 유저를 왜래키로 설정한다.

In [ ]:
아마 추측하기로는 settings.py에 이걸 추가하면 해결될 듯 싶다.
AUTH_USER_MODEL = 'yourapp.CustomUser'